In [1]:
#Libraries

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os
import re
import json
import string
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
from tqdm.autonotebook import tqdm
from functools import partial

In [8]:
!unzip coleridgeinitiative-show-us-the-data.zip -d coleridgeinitiative-show-us-the-data

Streaming output truncated to the last 5000 lines.
  inflating: coleridgeinitiative-show-us-the-data/train/a7a24e74-5bfe-43e0-a34d-82706221ab2c.json  
  inflating: coleridgeinitiative-show-us-the-data/train/a7a7d2a5-ba15-43f4-b832-128a4e3ad8c7.json  
  inflating: coleridgeinitiative-show-us-the-data/train/a7ac93e4-dfe4-4f34-ba08-977114203b3a.json  
  inflating: coleridgeinitiative-show-us-the-data/train/a7ae73f5-7b17-43d7-b8f4-367980968b0e.json  
  inflating: coleridgeinitiative-show-us-the-data/train/a7b26e45-1939-4950-9707-3a2b06e021d0.json  
  inflating: coleridgeinitiative-show-us-the-data/train/a7beb2c6-c2b1-4ed5-9862-c0a229604536.json  
  inflating: coleridgeinitiative-show-us-the-data/train/a7c0c511-0c49-4a2f-8e78-26bde0deb96a.json  
  inflating: coleridgeinitiative-show-us-the-data/train/a7d18fef-611c-42f5-956b-14deb780d9c3.json  
  inflating: coleridgeinitiative-show-us-the-data/train/a7d1ee49-3b19-47a4-9918-77dff9eaf4e2.json  
  inflating: coleridgeinitiative-show-us-the-data

In [9]:
train = pd.read_csv('coleridgeinitiative-show-us-the-data/train.csv')
train.head()

,Id,pub_title,dataset_title,dataset_label,cleaned_label
0,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,The Impact of Dual Enrollment on College Degre...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
1,2f26f645-3dec-485d-b68d-f013c9e05e60,Educational Attainment of High School Dropouts...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
2,c5d5cd2c-59de-4f29-bbb1-6a88c7b52f29,Differences in Outcomes for Female and Male St...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
3,5c9a3bc9-41ba-4574-ad71-e25c1442c8af,Stepping Stone and Option Value in a Model of ...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study
4,c754dec7-c5a3-4337-9892-c02158475064,"Parental Effort, School Resources, and Student...",National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study


In [10]:
train_files_path = './coleridgeinitiative-show-us-the-data/train'
test_files_path = './coleridgeinitiative-show-us-the-data/test'
counter = 0
total_counter=0
def json_to_text(filename, train_files_path=train_files_path):
    global counter, total_counter
    total_counter+=1
    json_path = os.path.join(train_files_path, (filename+'.json'))
    value = ""
    with open(json_path, 'r') as f:
        json_decode = json.load(f)
        for data in json_decode:
            if data.get('section_title') == "Introduction":
                counter += 1
                value +=" " + data.get('text')
            if data.get('section_title') == "Abstract":
                counter += 1
                value +=" " + data.get('text')
    return value
    
tqdm.pandas()
train['text'] = train['Id'].progress_apply(json_to_text)
print(counter)
print(total_counter)


20778
19661


In [18]:
train.head()

,Id,pub_title,dataset_title,dataset_label,cleaned_label,text
0,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,The Impact of Dual Enrollment on College Degre...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,
1,2f26f645-3dec-485d-b68d-f013c9e05e60,Educational Attainment of High School Dropouts...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,
2,c5d5cd2c-59de-4f29-bbb1-6a88c7b52f29,Differences in Outcomes for Female and Male St...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,
3,5c9a3bc9-41ba-4574-ad71-e25c1442c8af,Stepping Stone and Option Value in a Model of ...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,Federal Reserve Bank of Richmond S1. Accounti...
4,c754dec7-c5a3-4337-9892-c02158475064,"Parental Effort, School Resources, and Student...",National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,This article investigates an important factor...


In [24]:
test = 0
for i in range(len(train)):
    if train.loc[i][3] in train.loc[i][5]:
        test += 1
print(test)

4155


In [25]:
# filter on dataset_label appearing in text once
train['mask'] = train.apply(lambda x: x.dataset_label in x.text, axis=1)
df = train.loc[train["mask"] == True,:]

In [26]:
df

,Id,pub_title,dataset_title,dataset_label,cleaned_label,text,mask
4,c754dec7-c5a3-4337-9892-c02158475064,"Parental Effort, School Resources, and Student...",National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,This article investigates an important factor...,True
5,81b5f874-9b42-4d4f-8c50-b4bab24f8aed,The Gender Gap Reloaded: Are School Characteri...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,This study examines the wage gender gap of yo...,True
6,f35fdfa6-a3f9-4ef3-a858-c8e2efcd2349,The High School Environment and the Gender Gap...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,Despite the striking reversal of the gender g...,True
8,e5a4c6af-0854-4f7e-b088-06f03c18a01a,Locus of Control and Peer Relationships Among ...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,Abstract Past research has shown that locus o...,True
12,d5ff0ecd-ccf5-4f8a-a09c-182376eed4d0,Beyond the Laboratory: Evaluating the Survey E...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,"The black-white gap in achievement, as measur...",True
...,...,...,...,...,...,...,...
19613,d90a010d-8dba-4ffa-a2f1-473a45ddb8c7,Identification of potential inhibitors of SARS...,SARS-CoV-2 genome sequence,genome sequences of 2019-nCoV,genome sequences of 2019 ncov,A case of unidentified pneumonia was reported...,True
19618,a45b5269-9937-4f04-ad51-81e705d837c4,The importance of standardisation – COVID-19 C...,COVID-19 Image Data Collection,COVID-19 Image Data Collection,covid 19 image data collection,"At the end of 2019, a new coronavirus SARS-Co...",True
19621,bd180f8e-6dfa-4874-a172-957322f46867,COVID-Net: A Tailored Deep Convolutional Neura...,COVID-19 Image Data Collection,COVID-19 Image Data Collection,covid 19 image data collection,The COVID-19 pandemic continues to have a dev...,True
19625,118d2571-5938-4363-81d0-c3083d18a607,COVID-19 detection and disease progression vis...,COVID-19 Image Data Collection,COVID-19 Image Data Collection,covid 19 image data collection,The 2019 novel Coronavirus (COVID-19) has bec...,True


In [27]:
test = 0
unique = set()
for i in range(len(train)):
    test += 1
    unique.add(train.loc[i][2])
print(test)
print(len(unique))

19661
45


In [28]:
df.to_csv('data.csv', index=False)